In [60]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, BertModel
from transformers import Trainer, TrainingArguments
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
import numpy as np
from datasets import load_metric
import evaluate

In [24]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

In [25]:
input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sequence = "She made me angry when she was rude at dinner"

In [26]:
tokens = tokenizer(input_sentence, target_sequence, return_tensors='pt')

In [27]:
tokens

{'input_ids': tensor([[  101,  1153, 22296,  1143,  1114,  1123, 17073,  7640,   117, 24206,
           118,  9243,   117,  1105,  4267,  1116,  4894, 26426, 21047,  1120,
          1103,  4698,  4014,  1952,   102,  1153,  1189,  1143,  4259,  1165,
          1131,  1108, 14708,  1120,  4014,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [28]:
model(**tokens)

SequenceClassifierOutput(loss=None, logits=tensor([[0.1998, 1.0848]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [31]:
logits = model(**tokens).logits
logits

tensor([[0.1998, 1.0848]], grad_fn=<AddmmBackward0>)

In [35]:
result = torch.softmax(logits, dim=-1).tolist()[0]
torch.softmax(logits, dim=-1), F.softmax(logits, dim=-1)

(tensor([[0.2921, 0.7079]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.2921, 0.7079]], grad_fn=<SoftmaxBackward0>))

In [36]:
fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileid) for fileid in fileids[::2]]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids[::2]]

In [37]:
len(reviews)

1000

In [38]:
label_dict = {'neg':0, 'pos':1}
y = np.array([label_dict[c] for c in categories])
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [51]:
X_train_val, X_test, y_train_val, y_test = train_test_split(reviews, y, test_size=0.3, random_state=0)
len(X_train_val), len(X_test)

(700, 300)

In [40]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [41]:
batch_size = 10
y_pred = []
num_batch = len(y_test) // batch_size

In [42]:
for i in range(num_batch):
    inputs = tokenizer(X_test[i*batch_size:(i+1)*batch_size], truncation=True, padding=True, return_tensors='pt')
    inputs = inputs.to(device)
    logits = model(**inputs).logits
    pred = torch.softmax(logits, dim=-1)
    results = pred.cpu().detach().numpy().argmax(axis=1)
    y_pred += results.tolist()

torch.cuda.empty_cache()

In [46]:
y_test == np.array(y_pred)

array([False,  True,  True, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True, False,  True,  True,  True, False,  True,  True, False,
        True,  True, False, False,  True, False,  True, False,  True,
        True,  True,  True, False,  True,  True,  True, False,  True,
        True,  True,

In [44]:
score = sum(y_test == np.array(y_pred)) / len(y_test)
score

0.8066666666666666

In [47]:
sentence1 = "What a beautiful day!"
sentence2 = "Nvidia Titan XP has 12GB of VRAM"

In [48]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [49]:
tokens = tokenizer([sentence1, sentence2], padding=True, return_tensors='pt')
tokens

{'input_ids': tensor([[  101,  2054,  1037,  3376,  2154,   999,   102,     0,     0,     0,
             0,     0,     0],
        [  101,  1050, 17258,  2401, 16537, 26726,  2038,  2260, 18259,  1997,
         27830,  3286,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [50]:
tokens2 = tokenizer(sentence1, sentence2, padding=True, return_tensors='pt')
tokens2

{'input_ids': tensor([[  101,  2054,  1037,  3376,  2154,   999,   102,  1050, 17258,  2401,
         16537, 26726,  2038,  2260, 18259,  1997, 27830,  3286,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [52]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=0)
len(X_train), len(X_val), len(X_test)

(560, 140, 300)

In [62]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [63]:
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors='pt')
val_input = tokenizer(X_val, truncation=True, padding=True, return_tensors='pt')

In [64]:
train_input['input_ids'].shape

torch.Size([560, 512])

In [82]:
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).clone().detach() for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx]).clone().detach().long()
        return item
    def __len__(self):
        return len(self.labels)

In [83]:
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)

In [84]:
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [85]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=2,
                                  per_device_train_batch_size=8, per_device_eval_batch_size=8)
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, compute_metrics=compute_metrics)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [86]:
trainer.train()

c:\Users\admin\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 560
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 140
  Number of trainable parameters = 109483778


  0%|          | 0/140 [00:00<?, ?it/s]

C:\Users\admin\AppData\Local\Temp\ipykernel_316\1974908807.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]).clone().detach() for key, val in self.inputs.items()}


In [ ]:
trainer.evaluate(eval_dataset=val_dataset)